In [ ]:
import pandas, collections
amigo = pandas.read_csv('listsOfRbps/rna_binding_genes_from_amigo.txt', sep='\t', comment='#', header=1)
amigo = pandas.read_csv('old_pma_analysis_inputs/rna_binding_genes_from_amigo', sep='\t', comment='#')#, header=1)
mappings = pandas.read_csv('old_pma_analysis_inputs/id_mappings.txt', sep='\t')
print(mappings)
uni_to_ensg = dict(zip(mappings['Swissprot Id'], mappings['Ensembl Gene ID']))
amigo['_id'] = [x.split(':')[-1] for x in amigo['Uniprot']]
uniprot_to_id = dict(zip(amigo._id, amigo['Gene name']))
#uniprot = set(uniprot)

ensg_domain = pandas.read_csv('/Users/dp/misc_projects/perturb/translations/ensg_to_protein_domain.txt', sep='\t')
#ensg_to_domain = dict(zip(ensg_domain))
uni_to_domain = {}
#for k, v in uni_to_ensg.items():
#    if k in uni
print(ensg_domain)
print(amigo)

In [ ]:

class domainGetter:
    
    def __init__(self):
        self.rbd_domains = ['RRM_dom', 'Nucleotide-bd_a/b_plait', 'Piwi', 
         'dsRBD_dom', 'S1-like_RNA-bd_dom', 'RNA_helicase','DEAD',
         'Znf_CHHC','Znf_CCCH','DEATH','PUM-HD','KH_dom','LSM_dom', 'Znf_CCHC',
                      'PWI_dom']

        mappings = pandas.read_csv('old_pma_analysis_inputs/id_mappings.txt', sep='\t')
        print(mappings)
        uni_to_ensg = dict(zip(mappings['Swissprot Id'], mappings['Ensembl Gene ID']))
        

        ensg_id = pandas.read_csv('old_pma_analysis_inputs//ensg_enst_refseq_name.txt', sep='\t')
        ensg_id.drop_duplicates(subset=['Ensembl Gene ID'], inplace=True)
        self.ensg_to_id = dict(zip(ensg_id['Ensembl Gene ID'], ensg_id['Associated Gene Name']))
        
        domains = pandas.read_csv('./old_pma_analysis_inputs/ensg_to_protein_domain.txt', sep='\t')
        domains['Gene name'] = [self.ensg_to_id.get(x) for x in domains['Ensembl Gene ID']]

        self.ensg_to_domains = collections.defaultdict(set)
        for row in domains.to_dict('records'):
            if type(row['Interpro Short Description']) == type(''):
                self.ensg_to_domains[row['Ensembl Gene ID']].add(row['Interpro Short Description'])
                #print(self.ensg_to_domains)
                #raise IOError
                
        self.uni_to_domains = collections.defaultdict(set)
        for uniprot_id, ensg in uni_to_ensg.items():
            if ensg in self.ensg_to_domains:
                self.uni_to_domains[uniprot_id] = self.ensg_to_domains[ensg]
                
    def has_rbd(self, _iterable) -> bool:
        if type(_iterable) == type(""):
            _iterable = _iterable.split(',')

        for x in _iterable:
            for keyword in self.rbd_domains:
                if keyword in x:
                    return True
        return False
    
dg = domainGetter()
has_an_rbp = [dg.has_rbd(x) for x in dg.ensg_to_domains.values()]
#print(has_an_rbp)
print(f'{sum(has_an_rbp)}/{len(has_an_rbp)}')
#list(dg.uni_to_domains.keys())[0]

In [ ]:
0.97**30

In [ ]:

dancr = ['P06744', 'Q96EY7', 'P13646', 'P13797', 'Q15084', 'P30041', 'Q99832', 'Q8IXQ5', 'P06241', 'O60292', 'Q13277', 'P31948', 'P40926', 'P30086', 'P30048', 'O43707', 'P15311', 'P61604', 'P22234', 'P60174', 'P68400', 'P00558', 'Q9UHB6', 'O75369', 'P08758', 'P22314', 'P17302', 'Q96PY5']
rbps = [x for x in dancr if x in uniprot]

# DANCR
df = pandas.read_excel('/Users/dp/Downloads/48391_0_other_1_pz8jnp/48391_0_supp_498877_py4xqk.xlsx',
                         sheet_name='sig')

# XIST
df = pandas.read_excel('/Users/dp/Downloads/48391_0_other_1_pz8jnp/48391_0_supp_498878_py4xqf.xlsx',
                         sheet_name='sig')

if 'Acccession' in df.columns:
    df['Accession'] = df['Acccession']
rbps = df#.loc[[x in uniprot for x in df['Acccession']], :]
rbps['domains'] = [', '.join(dg.uni_to_domains.get(x, '')) for x in rbps.Accession]
rbps['RBP?'] = [dg.has_rbd(x) for x in rbps.domains]
print(rbps.loc[26]['domains'])
print(rbps)
print(dancr)